In [1]:
import pynq
import dishsoap
import numpy as np

dish = dishsoap.Overlay()

**Simulate the simple network for 5 steps. Start with:**
- *TF* set to `1`
- All other elements set to `0`

In [ ]:
results = dish.run_synch([1,0,0,0,0], 5)
for i, name in enumerate(dish.elements):
    print(f'{name:10}', results[i])

**How long does it take to simulate this network for 5000 steps?**

In [ ]:
%%timeit results = pynq.allocate(shape=(5001,),dtype=np.uint64)
dish.run_synch_inplace([0,1,0,0,0], results)

**How does runtime scale with iteration counts?**

In [ ]:
max_exp = 15
results = pynq.allocate(shape=(max_exp + 1, 2**max_exp), dtype=np.uint64)
for exp in range(max_exp + 1):
    print('Running', 2**exp, 'iterations:')
    %timeit dish.run_synch_inplace([1,0,0,0,0], results[exp,:2**exp])
    

In [ ]:
results = pynq.allocate(shape=(1001,),dtype=np.uint64)

In [ ]:
%time dish.run_synch_inplace([1,1,1], results)

In [ ]:
x = results[:1000]

In [ ]:
x.shape

In [ ]:
dish.unpack(results)

In [ ]:
results.shape

In [ ]:
# generate all network states
all_init_states = np.arange(2**dish.network_size, dtype=np.uint64)

# for TReg network, pick only a few initial states ;)
#all_init_states = np.array([0, 40, 5013, 108513, 13], dtype=np.uint64)

for init in all_init_states:
    # pretty-print the network state
    print('init state:', dish.unpack(init))
    # run each network state for some number of iterations
    results = dish.run_synch_np(init, 5)
    result_array = np.array([dish.unpack(r) for r in results])
    print(result_array)

In [ ]:
init_state = dishsoap.list_to_packed([1,1,0])
results = pynq.allocate(shape=(2000,), dtype=np.uint64)
%time dish.run_synch_inplace(init_state, results)
print('number states saved:', len(results))
print('first two results:\r\n', dish.unpack(results[:2]))
print('last two results:\r\n',  dish.unpack(results[-2:]))

In [ ]:
%%time 
def benchmark(init_states, iterations = 5000):
    print('---allocating all memory---')
    n_states = len(init_states)
    %time pynq.allocate(shape=(iterations*n_states,), dtype=np.uint64)
    results = pynq.allocate(shape=(iterations,n_states), dtype=np.uint64)
    print('---running DiSH---')
    for s, r in zip(init_states, results):
        %time dish.run_synch_inplace(s, r)
    print('---total time---')
    # show first and last, prove we did it all
    #print(dish.unpack(results[:n_states,:1]))
    #print(dish.unpack(results[:n_states,-1:]))
    
benchmark(all_init_states)

In [2]:
# do one where we just scale up each time
def benchmark(init_state, results):
    dish.run_synch_inplace(init_state, results)
    # to be fair to DiSH, let's also write the output in the same format:
    iterations = results.shape[0]
    with open(f'bench-gene-expression-dishsoap-{iterations}.txt', 'w') as f:
        f.write('# time ')
        f.write(' '.join(dish.elements))
        f.write(' step\n')
        
        for step, vals in enumerate(dish.unpack(results)):
            f.write(f'{step}  ')
            f.write(' '.join([str(v) for v in vals]))
            f.write(f' {step}\n')

steps = range(16)
for iterations in [2**s for s in steps]:
    results = pynq.allocate(shape=(iterations,), dtype=np.uint64)
    print(f'Running: {iterations:7d} ', end='')
    %timeit benchmark([1,0,0,0,0], results)

Running:       1 3.71 ms ± 1.95 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       2 3.43 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       4 3.52 ms ± 365 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       8 4.52 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      16 7.02 ms ± 437 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      32 13.8 ms ± 44.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      64 22.7 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:     128 40 ms ± 201 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:     256 76.2 ms ± 460 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:     512 147 ms ± 712 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:    1024 292 ms ± 9.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Running:    2048 581 ms ± 9.18 ms per l

# Begin DiSH Benchmarks

In [2]:
from dish.simulator import Simulator

model = Simulator('tcell-regulatory.xlsx')

def benchmark(iterations: int):
    model.run_simulation('sync', 1, iterations, f'bench-tcell-regulatory-{iterations}.txt', 0, 2)

steps = range(16)
for iterations in [2**s for s in steps]:
    print(f'Running: {iterations:7d} ', end='')
    %timeit benchmark(iterations)

Running:       1 1.06 ms ± 8.66 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Running:       2 1.94 ms ± 7.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
Running:       4 3.73 ms ± 63.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       8 7.28 ms ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      16 14.3 ms ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      32 28.5 ms ± 531 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:      64 56.1 ms ± 902 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:     128 117 ms ± 2.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:     256 231 ms ± 4.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Running:     512 454 ms ± 8.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Running:    1024 906 ms ± 7.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Running:    2048 1.79 s ± 3.49 ms p

In [11]:
# now be unfair to DiSH: show off a smarter way to save our run data
import pickle, gzip

def benchmark(init_state, results):
    dish.run_synch_inplace(init_state, results)
    # write the output in a much more effective format
    iterations = results.shape[0]
    with gzip.open(f'bench-gene-expression-dishsoap-{iterations}.pickle.gz', 'wb') as f:
        data = {
            'elements': dish.elements,
            'results': results,
        }
        pickle.dump(data, f)

steps = range(16)
for iterations in [2**s for s in steps]:
    results = pynq.allocate(shape=(iterations,), dtype=np.uint64)
    print(f'Running: {iterations:7d} ', end='')
    %timeit benchmark([1,0,0,0,0], results)

Running:       1 2.68 ms ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Running:       2 2.68 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       4 2.97 ms ± 686 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:       8 4.44 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      16 2.71 ms ± 18.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      32 2.75 ms ± 143 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:      64 2.8 ms ± 94.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:     128 2.89 ms ± 323 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:     256 2.87 ms ± 82.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:     512 3.4 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:    1024 3.38 ms ± 74.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Running:    2048 4.73 ms ± 

In [7]:
unpickled = None
with gzip.open('bench-gene-expression-dishsoap-16.pickle.gz', 'rb') as f:
    unpickled = pickle.load(f)

unpickled

{'elements': ['Inh', 'TF', 'Xgene', 'Xprotein', 'Xrna'],
 'results': PynqBuffer([ 1,  5, 13, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,
             29], dtype=uint64)}

In [10]:
dish.elements

['TF', 'Inh', 'Xgene', 'Xrna', 'Xprotein']